注意：
- 如果你使用vscode运行本笔记本，请为vscode安装Jupyter Notebook Renderers插件，否则看不到可视化plotly的输出
- 如果使用jupyter lab或notebook启动服务器查看，应该是能够正常预览的。如果看不到，请安装"jupyterlab>=3","notebook>=5.3","ipywidgets>=7.6"

In [127]:
%pip install plotly==5.15.0 

Note: you may need to restart the kernel to use updated packages.


In [128]:
import sqlite3
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import display
import plotly.express as px
from pprint import pprint

In [129]:
# example_df = pd.read_csv('./data/adult.data')
# example_df.head()

先对数据做一些清洗，删除掉包含NaN的行。

In [130]:


columns = [
"age",  # 年龄, int
"workclass",  # 雇主类型、雇佣类型，离散
"fnlwgt", #人口普查员认为观测值的人数，int
"education",  # 教育程度，离散
"education-num", # 受教育年限，int
"marital-status",  # 婚姻状况，离散
"occupation",  # 职业，离散
"relationship",  #家庭角色，离散
"race",  # 种族， 离散
"sex", # 性别， 离散
"capital-gain", # 资本收益， 数值
"capital-loss", # 资本损失， 数值
"hours-per-week", # 每周工作时间， int
"native-country", # 原生国籍，离散 
"income" # 收入是否大于50K， 布尔
]


def preprocessing_data():
    df = pd.read_csv('./data/adult.data', header=None)

    df.columns = columns

    # 进行数据清洗，过滤掉为取值为 ? 的数据
    # 第一步需要先去掉所有value值中空格
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    # 统计过滤前的数据量
    # print("adult_df过滤前的数据量：", df.shape[0])

    # 将 ? 替换为 np.nan
    df = df.replace("?", np.nan)
    df = df.replace(" ", np.nan)
    df = df.replace("", np.nan)

    # 过滤掉为 np.nan 的数据
    df_filter = df.dropna()

    nan_columns = df_filter.columns[df_filter.isnull().any()]
    # print("包含NaN值的列：", nan_columns)

    # 打印包含NaN值的行
    nan_rows = df_filter[df_filter.isnull().any(axis=1)]
    # print("包含NaN值的行：")
    # print(nan_rows)

    # 统计过滤后的数据量
    # print("adult_df过滤后的数据量：", df_filter.shape[0])

    # # 统计异常数据占比（按照百分比输出）
    # print("adult_df异常数据占比：", (1 - df_filter.shape[0] / df.shape[0]) * 100, "%")

    # print(df_filter.columns)
    # print(df_filter["income"].unique())
    # print(df_filter["native-country"].unique())
    # print(df_filter['age'].unique())

    # 将salary列的值转换为bool值
    # df["income>50K"] = np.where(df["income"] == ' <=50K', False, True)
    # df.drop("income", axis=1, inplace=True)

    # display(df_filter)
    # pprint(df_filter['occupation'].unique())
    return df_filter

adult_df = preprocessing_data()


在第一个可视化，我们使用平行坐标图，展示职业（occupation）和雇主类型（workclass）对收入的影响。
- 采用的数据类型：职业（occupation）、雇主类型（workclass）、收入（income）。
- 选择此可视化和数据的原因：职业和雇主类型是和收入最直接相关的信息，希望能揭示他们之间的关系，确定目标人群的职业。
- 预期：高技能要求的职业和管理职业应该会比其他职业收入高。政府雇员和私企哪个收入高是很有意思的一点


In [131]:
import plotly.express as px
import plotly.graph_objects as go

paracate_df = adult_df.copy()

rendered_columns = [
# "age_bin",  # 年龄, int  转离散
"workclass",  # 雇主类型、雇佣类型，离散
# "fnlwgt", #人口普查员认为观测值的人数，int
# "education",  # 教育程度，离散
# "education-num", # 受教育年限，int， 转离散
# "edu_years_bin",
# "marital-status",  # 婚姻状况，离散
"occupation",  # 职业，离散
# "relationship",  #家庭角色，离散
# "race",  # 种族， 离散
# "sex", # 性别， 离散
# "capital-gain", # 资本收益， 数值
# "capital-loss", # 资本损失， 数值
# "hours-per-week", # 每周工作时间， int  转离散
# "work_hour_bin",
# "native-country", # 原生国籍，离散 
"income" # 收入是否大于50K， 布尔
]

continuous_cols = [
"age",  # 年龄, int  转离散
"education-num", # 受教育年限，int， 转离散
"hours-per-week", # 每周工作时间， int  转离散
]

paracate_df["income>50K"] = np.where(paracate_df["income"] == ' <=50K', 0, 1)
# display(df.head())

# dims
workclass_dim = go.parcats.Dimension(
    values=paracate_df['workclass'],
    categoryorder='category ascending', label="workclass"
)
occupation_dim = go.parcats.Dimension(values=paracate_df['occupation'], label="occupation")
income_dim = go.parcats.Dimension(values=paracate_df['income'], label="income")

# line color
color = paracate_df.income;
color_map = paracate_df.income.map({'<=50K': 'lightsteelblue', '>50K':'mediumseagreen'})
colorscale = [['<=50K', 'lightsteelblue'], ['>50K', 'mediumseagreen']];

# draw
fig = go.Figure(data = [go.Parcats(dimensions=[workclass_dim, occupation_dim, income_dim],
        line={'color': color_map},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

# fig = px.parallel_categories(data_frame=df.head(10000), dimensions=rendered_columns, color="education-num")

# fig = px.parallel_categories(data_frame=df.head(100), dimensions=rendered_columns, color='age')

fig.update_layout(
    height=1000,
    width=500
)

fig.show()

# 移动到图上，可以看到统计数据

评述和总结：上面的平行坐标图，符合之前的预期，高技能要求的职业和管理职业的收入确实比其他职业高。

下面是具体分析：
- 按雇主/雇佣类型分类
    - 私企人数最多，是一个比较大的群体，适合作为课程的目标人群
    - 政府雇员（包括联邦雇员、州雇员、本地雇员）的高收入人群比例更高，他们的收入更稳定，但是人数较少
    - 自雇人群(self-emp-inc和self-emp-not-inc)的高收入人群比例也很高，他们主要是自我雇佣的医生、律师、建筑师、咨询顾问、小企业所有者、自雇专业人员、自我雇佣的艺术家、作家和表演者、自我雇佣的技工等等。他们的收入较高，但是人数较少。可以针对性地为他们开发高单价的课程
- 按职业分类
    - 管理人员、专业技术人员的高收入人群比例较高。我们可以为其开发高单价的课程
    - 行政办公室人员、保洁人员、销售人员、技术支持人员、手工艺人、渔牧人员、机器操控人员等等的高收入人群比例较低。适合为其开发低单价的课程



=======================================================================================================================================


第二个可视化，我们使用树状图（treemap），展示职业（occupation）和出生国籍（native-country）的关系，展现每种职业里国籍分布有多少。
- 采用数据类型：职业、出生国籍
- 选择此可视化和数据的原因：上一个可视化确定了目标职业，这一个可视化希望通过确定目标职业的主要国籍，来更好地制定营销策略，以更好地覆盖目标人群。
- 预期：不同国家的人在就业时有类似偏好，如中国人可能专业技术类职业较多。

In [132]:
import plotly.express as px
treemap_df = adult_df.copy()
df_without_us = treemap_df[treemap_df["native-country"] != "United-States"]

# display(treemap_df)
# path = [px.Constant("all"), 'native-country', 'occupation']
# pprint(treemap_df['occupation'].unique())

path = [px.Constant("all"), 'occupation', 'native-country']
fig = px.treemap(treemap_df, path=path, title="Occupation and native country")
fig.update_traces(root_color="lightgrey")
fig.show()

without_us_path = [px.Constant("without US"), 'occupation', 'native-country']
fig = px.treemap(df_without_us, path=without_us_path, title="Occupation and native country without US")
fig.update_traces(root_color="lightgrey")
fig.show()

# 点击treemap的区域，可以进入下一层。点击上一层返回

评述和总结：上面的树状图，总体上符合之前的预期，只是美国人在所有职业中都占大多数，所以我添加了一个排除美国人的树状图，来分析其他国家人在职业中的占比。我所预期的不同国家人的就业偏好不同，是符合预期的。

下面是具体分析：
- 专业技能工作者中，除美国人之外，印度、德国、菲律宾、加拿大、中国占比较多。说明了这些国家的人在专业技能方面的竞争力较强。我们可以在开发相关专业技能课程时，优先提供印度、德国、菲律宾、中国等国家语言的字幕，并且营销上覆盖到这些国家的人群。
- 高管中，除美国人之外，英格兰、德国、日本、加拿大 等传统发达国家占比较多，并且古巴、墨西哥人也有一定比例。我们可以在开发管理类课程时，优先提供德国、日本、古巴、墨西哥等国的语言字幕，并且营销上覆盖到这些国家的人群。
- 墨西哥人在多个较低收入的职位中占比为第一，如手工业者、机器操作员、行政文书岗位、清洁工、渔牧业、交通运输业、销售、私人安保等等。说明了墨西哥因为和美国接壤，偷渡过来的低收入群体较多。我们可以以墨西哥人为目标，为他们提供相应的职业培训，以提高他们的收入


===================================================================================================================================================

第三个可视化，原打算是使用基于像素的图，展示职业（occupation）和教育程度（education）的关系，后面换成了堆叠条形图。
- 采用数据类型：职业、教育程度。
- 选择此可视化和数据的原因：在获得高收入职业之后，通过挖掘职业和教育程度之间的关系，我们可以得到目标职业的大部分人受教育程度是多少，以此推出相应下一阶段的教育课程，比如如果一个群体大部分人是高中毕业水平，那么推出大学水平课程就会比较受欢迎。另外我们也可以得到低教育群体的在哪些职业占比多，以推出相应的职业课程，提高他们的受教育程度，以提升他们的竞争力。
- 预期：高收入群体的教育程度应该比较高

In [133]:
# occupation, education 
from pandas.api.types import CategoricalDtype

bar_df = adult_df.copy()
edu_order = ['Preschool','1st-4th','5th-6th','7th-8th','9th', '10th', '11th', '12th','HS-grad','Some-college', 'Assoc-acdm', 'Assoc-voc','Prof-school', 'Bachelors', 'Masters','Doctorate']

# 创建教育程度分类
cat_type = CategoricalDtype(categories=edu_order, ordered=True)

# 对 occupation 和 education 列分组并计数，然后重命名 ‘occupation' 列并重新分类 'education' 列，最后按照 'occupation' 和 'education' 列对结果进行排序
count_df = bar_df.groupby(['occupation', 'education'])['occupation'].count().reset_index(name='count')
count_df['education'] = count_df['education'].astype(cat_type)
count_df = count_df.sort_values(['occupation', 'education'])

total_df = count_df.groupby('occupation', as_index=False)['count'].agg({'total': 'sum'})
def calculate_percentage(count_df, total_df):
    percentage_list = []
    for index, row in count_df.iterrows():
        occupation = row['occupation']
        education = row['education']
        count = row['count']
        total_count = total_df.loc[total_df['occupation'] == occupation, 'total'].iloc[0]
        percentage = count / total_count * 100
        percentage_list.append(percentage)

    count_df['percentage'] = percentage_list
    return count_df

count_df = calculate_percentage(count_df, total_df)

# display(count_df)

fig = px.bar(count_df, x="occupation", y="percentage", color="education", title="Education category in each occupation", height=900)
fig.show()

# 移动到图上，可以看到education类别、occupation类别和百分比

评述和总结：在分析过程中，我发现像素图不适合用来可视化教育程度和职业，因为教育程度的类型较多，有16个，用像素图用户会迷失在不同的教育类型中。而如果使用堆叠柱状图，因为它的顺序性更强，更加契合教育程度的天然顺序性，便于比较。

下面是具体分析：
- 教育程度最高的（大量学士、一些硕士和博士）：专业技术人员和高管。他们是较重视教育的人群，根据之前的分析，他们的收入也是最高的，所以我们可以为他们开发高单价高教育水平的课程，例如硕士和博士课程。
- 教育程度中等的（大量高中毕业、一些学士、少量硕士）：安保警察（protective-serv）、销售、技术支持、军队（Armed-Forces）。可以为他们开发学士和硕士课程。
- 教育程度较低的（大量高中毕业、职业学校、少量学士）：行政文书、清洁工、机器操作员、手工业者、渔牧业、交通运输业、私人住宅服务。可以为他们开发学士或职业学校课程。